In [ ]:
import pandas as pd
import pymysql
import json

# Loading CSV files into DataFrames using pandas

In [11]:
customers = pd.read_csv("customers.csv")
accounts = pd.read_csv("accounts.csv")
transactions = pd.read_csv("transactions.csv")
loans = pd.read_csv('loans.csv')
branches = pd.read_csv('branches.csv')
support_tick = pd.read_csv('support_tickets.csv')

In [18]:
dfs = {
    "Customers": customers,
    "Accounts": accounts,
    "Transactions": transactions,
    "Loans": loans,
    "Support Tickets": support_tick,
    "Branches": branches
}

# Describing DataFrames

In [38]:
for name, df in dfs.items():
    print(f"--- {name} ---")
    print(df.describe())
    print("\n")

--- Customers ---
              age
count  500.000000
mean    44.398000
std     15.255985
min     18.000000
25%     31.000000
50%     45.000000
75%     57.000000
max     70.000000


--- Accounts ---
       account_balance
count       500.000000
mean     246996.117180
std      143128.631994
min        1669.730000
25%      127210.265000
50%      259209.460000
75%      370645.007500
max      498739.490000


--- Transactions ---
             amount
count  10000.000000
mean   49721.848516
std    29045.696434
min      106.440000
25%    24568.575000
50%    49323.930000
75%    74957.352500
max    99999.050000


--- Loans ---
          Loan_ID  Customer_ID   Account_ID   Loan_Amount  Interest_Rate  \
count  553.000000   553.000000   553.000000  5.530000e+02     553.000000   
mean   277.000000  1305.819168  2318.987342  2.472737e+06      10.528933   
std    159.781622   170.117382   171.044134  1.410436e+06       2.381320   
min      1.000000  1002.000000  2004.000000  5.965500e+04       6.50000

# Info of each DataFrames

In [31]:
for name, df in dfs.items():
    print(f"--- {name} ---")
    print(df.info())
    print("\n")

--- Customers ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   customer_id   500 non-null    object
 1   name          500 non-null    object
 2   gender        500 non-null    object
 3   age           500 non-null    int64 
 4   city          500 non-null    object
 5   account_type  500 non-null    object
 6   join_date     500 non-null    object
dtypes: int64(1), object(6)
memory usage: 27.5+ KB
None


--- Accounts ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      500 non-null    object 
 1   account_balance  500 non-null    float64
 2   last_updated     500 non-null    object 
dtypes: float64(1), object(2)
memory usage: 11.8+ KB
None


--- Transactions ---
<class 'pa

# Checking for duplicate values

In [32]:
for name, df in dfs.items():
    print(f"--- {name} ---")
    print(df.duplicated().any())
    print("\n")

--- Customers ---
False


--- Accounts ---
False


--- Transactions ---
False


--- Loans ---
False


--- Support Tickets ---
False


--- Branches ---
False




In [40]:
for name, df in dfs.items():
    print(f"--- {name} ---")
    print(df.isnull().sum())
    print("\n")

--- Customers ---
customer_id     0
name            0
gender          0
age             0
city            0
account_type    0
join_date       0
dtype: int64


--- Accounts ---
customer_id        0
account_balance    0
last_updated       0
dtype: int64


--- Transactions ---
txn_id         0
customer_id    0
txn_type       0
amount         0
txn_time       0
status         0
dtype: int64


--- Loans ---
Loan_ID             0
Customer_ID         0
Account_ID          0
Branch              0
Loan_Type           0
Loan_Amount         0
Interest_Rate       0
Loan_Term_Months    0
Start_Date          0
End_Date            0
Loan_Status         0
dtype: int64


--- Support Tickets ---
Ticket_ID               0
Customer_ID             0
Account_ID              0
Loan_ID               371
Branch_Name             0
Issue_Category          0
Description             0
Date_Opened             0
Date_Closed           104
Priority                0
Status                  0
Resolution_Remarks      0
S

In [41]:
support_tick.shape

(600, 15)

In [42]:
support_tick['Loan_ID'] = support_tick['Loan_ID'].fillna(0)
# Null values for Loan_Id are cleared

In [43]:
support_tick.isnull().sum() 

Ticket_ID               0
Customer_ID             0
Account_ID              0
Loan_ID                 0
Branch_Name             0
Issue_Category          0
Description             0
Date_Opened             0
Date_Closed           104
Priority                0
Status                  0
Resolution_Remarks      0
Support_Agent           0
Channel                 0
Customer_Rating         0
dtype: int64

# Connector

In [45]:
connection = pymysql.connect(
    host= 'localhost',
    user= 'root',
    password='root',    
)
cursor = connection.cursor()

In [46]:
Create_DB = """CREATE DATABASE IF NOT EXISTS BANK"""
cursor.execute(Create_DB)
print("Database Created successfully")

Database Created successfully


In [ ]:
#Customer table
Create_Tab = """
        CREATE TABLE IF NOT EXISTS BANK.CUSTOMERS
        (
            CUSTOMER_ID VARCHAR(10) PRIMARY KEY,
            NAME VARCHAR(50),
            GENDER CHAR(1),
            AGE INT,
            CITY VARCHAR(50),
            ACCOUNT_TYPE VARCHAR(20),
            JOIN_DATE DATE  
        )
"""
cursor.execute(Create_Tab)

0

In [ ]:
# inserting records from DataFrames to SQL Table
for index, rows in customers.iterrows():
    cursor.execute("""
                   INSERT INTO BANK.CUSTOMERS(`customer_id`, `name`, `gender`, `age`, `city`, `account_type`,`join_date`)
                    VALUES (%s,%s,%s,%s,%s,%s,%s)""",
                    (rows['customer_id'], rows['name'], rows['gender'],rows['age'],rows['city'],rows['account_type'],rows['join_date']))

connection.commit()


IntegrityError: (1062, "Duplicate entry 'C0001' for key 'customers.PRIMARY'")

In [ ]:
# Records succesfully inserted
cursor.execute("""SELECT * FROM BANK.CUSTOMERS""")
rows = cursor.fetchall()
for row in rows:
    print(row)

('C0001', 'Richard Wilson', 'M', 65, 'Kimberlyburgh', 'Savings', datetime.date(2023, 8, 27))
('C0002', 'Holly Parker', 'F', 41, 'Port David', 'Current', datetime.date(2023, 12, 8))
('C0003', 'Megan Alvarez', 'M', 46, 'West Laurieton', 'Savings', datetime.date(2022, 5, 25))
('C0004', 'Nicholas Stewart', 'F', 39, 'Amyville', 'Current', datetime.date(2020, 11, 30))
('C0005', 'Douglas Anderson', 'M', 70, 'New Craigport', 'Current', datetime.date(2023, 1, 29))
('C0006', 'Christopher Burton', 'F', 70, 'North Cody', 'Savings', datetime.date(2023, 6, 12))
('C0007', 'Jasmine Morris', 'M', 69, 'Kristaborough', 'Savings', datetime.date(2023, 10, 9))
('C0008', 'Mark Cooper', 'F', 30, 'Port Jenniferton', 'Current', datetime.date(2023, 4, 25))
('C0009', 'Christopher Smith', 'M', 66, 'Port Nathan', 'Savings', datetime.date(2020, 8, 17))
('C0010', 'Connie Rodriguez', 'F', 56, 'Banksburgh', 'Savings', datetime.date(2023, 6, 21))
('C0011', 'Carolyn Higgins', 'M', 26, 'Kinghaven', 'Savings', datetime.dat

In [ ]:
# Creating Accounts Table
account = """
        CREATE TABLE IF NOT EXISTS BANK.ACCOUNTS
        (
            CUSTOMER_ID VARCHAR(10), 
            ACCOUNT_BALANCE FLOAT, 
            LAST_UPDATED DATE, 
            FOREIGN KEY (CUSTOMER_ID) REFERENCES CUSTOMERS(CUSTOMER_ID)
        )
"""

cursor.execute(account)

# Inserting Values into Accounts Table
for index, rows in accounts.iterrows():
    cursor.execute("""
                   INSERT INTO BANK.ACCOUNTS(`customer_id`, `account_balance`, `last_updated`)
                   VALUES (%s,%s,%s)""",
                   (rows['customer_id'], rows['account_balance'], rows['last_updated']))
    
connection.commit()

0

In [ ]:
# Creating Transactions Table
transaction = """
            CREATE TABLE IF NOT EXISTS BANK.TRANSACTIONS
            (
                TXN_ID VARCHAR(10) PRIMARY KEY, 
                CUSTOMER_ID VARCHAR(10), 
                TXN_TYPE VARCHAR(15), 
                AMOUNT FLOAT, 
                TXN_TIME DATETIME, 
                STATUS VARCHAR(15), 
                FOREIGN KEY (CUSTOMER_ID) REFERENCES CUSTOMERS(CUSTOMER_ID)
            )
"""

cursor.execute(transaction)

# Inserting Values into Transactions Table
for index, rows in transactions.iterrows():
    cursor.execute("""
                   INSERT INTO BANK.TRANSACTIONS(`txn_id`,`customer_id`, `txn_type`, `amount`,`txn_time`,`status`)
                   VALUES (%s,%s,%s,%s,%s,%s)""",
                   (rows['txn_id'],rows['customer_id'], rows['txn_type'], rows['amount'],rows['txn_time'],rows['status']))
    
connection.commit()

In [ ]:
# Creating Loans Table
cursor.execute("""
               CREATE TABLE IF NOT EXISTS BANK.LOANS
               (
                  LOAN_ID INT PRIMARY KEY, 
                  CUSTOMER_ID VARCHAR(10), 
                  ACCOUNT_ID VARCHAR(15), 
                  BRANCH VARCHAR(100), 
                  LOAN_TYPE VARCHAR(30), 
                  LOAN_AMOUNT INT, 
                  INTEREST_RATE FLOAT,
                  LOAN_TERM_MONTHS INT,
                  START_DATE DATE, 
                  END_DATE DATE, 
                  LOAN_STATUS VARCHAR(30)
               )
""")

#Inserting Values into Loans Table
for index, rows in loans.iterrows():
    cursor.execute("""
                  INSERT INTO BANK.LOANS(`LOAN_ID`,`CUSTOMER_ID`, `ACCOUNT_ID`, `BRANCH`,`LOAN_TYPE`,`LOAN_AMOUNT`,`INTEREST_RATE`,`LOAN_TERM_MONTHS`,`START_DATE`,`END_DATE`,
                  `LOAN_STATUS`)
                  VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""",
                  (rows['Loan_ID'],rows['Customer_ID'], rows['Account_ID'], rows['Branch'],rows['Loan_Type'],rows['Loan_Amount'],rows['Interest_Rate'],
                   rows['Loan_Term_Months'],rows['Start_Date'],rows['End_Date'],rows['Loan_Status']))

connection.commit()

0

In [ ]:
#Branches Table

cursor.execute("""
               CREATE TABLE IF NOT EXISTS BANK.Branches
               (
                   Branch_ID INT PRIMARY KEY, 
                   Branch_Name VARCHAR(100), 
                   City VARCHAR(30), 
                   Manager_Name VARCHAR(50), 
                   Total_Employees INT, 
                   Branch_Revenue FLOAT, 
                   Opening_Date DATE, 
                   Performance_Rating INT
                )
""")

#Inserting Values into Branches Table

for index, rows in branches.iterrows():
    cursor.execute("""
                   INSERT INTO BANK.branches(`Branch_ID`,`Branch_Name`, `City`, `Manager_Name`,`Total_Employees`,`Branch_Revenue`,`Opening_Date`,`Performance_Rating`)
                   VALUES (%s,%s,%s,%s,%s,%s,%s,%s)""",
                   (rows['Branch_ID'], rows['Branch_Name'], rows['City'],rows['Manager_Name'],rows['Total_Employees'],rows['Branch_Revenue'],rows['Opening_Date'],
                    rows['Performance_Rating']))


connection.commit()

0

In [ ]:
# Creating Support_ticket Table
cursor.execute("""
               CREATE TABLE IF NOT EXISTS BANK.SUPPORT_TICK
               (
                   Ticket_ID VARCHAR(30) PRIMARY KEY, 
                   Customer_ID VARCHAR(30), 
                   Account_ID VARCHAR(30), 
                   Loan_ID VARCHAR(30), 
                   Branch_Name VARCHAR(50), 
                   Issue_Category VARCHAR(100),
                   Description VARCHAR(200), 
                   Date_Opened DATE NULL, 
                   Date_Closed DATE NULL, 
                   Priority VARCHAR(15), 
                   Status VARCHAR(15),
                   Resolution_Remarks VARCHAR(200), 
                   Support_Agent VARCHAR(30), 
                   Channel VARCHAR(10), 
                   Customer_Rating INT
                   )
""")


0

In [42]:
def clean_date(val):
    if pd.isnull(val) or val in ("", None):
        return None
    return val

In [ ]:
for index, rows in support_tick.iterrows():
    cursor.execute("""
                   INSERT INTO BANK.SUPPORT_TICK
                   (
                       `Ticket_ID`,`Customer_ID`,`Account_ID`,`Loan_ID`,`Branch_Name`,`Issue_Category`,`Description`,`Date_Opened`,`Date_Closed`,`Priority`,`Status`,
                       `Resolution_Remarks`,`Support_Agent`,`Channel`,`Customer_Rating`)
                       VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s
                    )""",
                    (
                        rows['Ticket_ID'],
                        rows['Customer_ID'],
                        rows['Account_ID'],
                        rows['Loan_ID'],
                        rows['Branch_Name'],
                        rows['Issue_Category'],
                        rows['Description'],
                        clean_date(rows['Date_Opened']),
                        clean_date(rows['Date_Closed']),
                        rows['Priority'],
                        rows['Status'],
                        rows['Resolution_Remarks'],
                        rows['Support_Agent'],
                        rows['Channel'],
                        rows['Customer_Rating'],
                        )
                    )
connection.commit()

# Creating and inserting values from JSON files

In [33]:
with open("credit_cards.json", "r") as file:
    credit_cards = json.load(file) #loading json file into a variable

In [ ]:
cursor.execute("""
               CREATE TABLE IF NOT EXISTS BANK.CREDIT_CARDS
               (
                   Card_ID INT PRIMARY KEY,
                   Customer_ID INT,Account_ID INT,
                   Branch VARCHAR(50),
                   Card_Number VARCHAR(20),
                   Card_Type VARCHAR(20),
                   Card_Network VARCHAR(15),
                   Credit_Limit INT,
                   Current_Balance FLOAT,
                   Issued_Date DATE,
                   Expiry_Date DATE,
                   Status VARCHAR(15)
                   )
            """)

0

In [ ]:
for records in credit_cards:
    cursor.execute("""
                   INSERT INTO BANK.credit_cards(`Card_ID`,`Customer_ID`,`Account_ID`,`Branch`,`Card_Number`,`Card_Type`,`Card_Network`,`Credit_Limit`,
                   `Current_Balance`,`Issued_Date`,`Expiry_Date`,`Status`) 
                    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""",
                    (records["Card_ID"],records['Customer_ID'],records['Account_ID'],records['Branch'],records['Card_Number'],records['Card_Type'],
                     records['Card_Network'],records['Credit_Limit'],records['Current_Balance'],records['Issued_Date'],records['Expiry_Date'],records['Status']))
connection.commit()

# Questions

In [48]:
quetions = {
        "Q1" : 
            """SELECT 
	                C.CITY, 
                    COUNT(DISTINCT C.CUSTOMER_ID) AS Total_Customers,
                    ROUND(AVG(A.ACCOUNT_BALANCE),2) AS Average_Balance
                FROM CUSTOMERS C 
                JOIN ACCOUNTS A ON C.CUSTOMER_ID = A.CUSTOMER_ID 
                GROUP BY C.CITY
                ORDER BY Average_Balance DESC""",
        "Q2" :
            """SELECT 
	                C.ACCOUNT_TYPE 
                FROM CUSTOMERS C
                JOIN ACCOUNTS A ON C.CUSTOMER_ID = A.CUSTOMER_ID
                GROUP BY ACCOUNT_TYPE
                HAVING MAX(A.ACCOUNT_BALANCE)
                ORDER BY C.ACCOUNT_TYPE 
                LIMIT 1""",
        "Q3" :
            """SELECT 
	            C.CUSTOMER_ID, 
                C.Name,
                ROUND(SUM(A.Account_Balance),2) AS Total_Balance 
            FROM ACCOUNTS A 
            JOIN CUSTOMERS C ON C.CUSTOMER_ID = A.CUSTOMER_ID
            GROUP BY C.CUSTOMER_ID
            ORDER BY Total_Balance DESC
            LIMIT 10""",
        "Q4" :
            """SELECT 
	            C.CUSTOMER_ID,
                C.Name
            FROM CUSTOMERS C
            JOIN ACCOUNTS A ON C.CUSTOMER_ID = A.CUSTOMER_ID 
            WHERE YEAR(C.JOIN_DATE) = 2023 AND A.ACCOUNT_BALANCE > 100000;""",
        "Q5" :
            """SELECT 
	            TXN_TYPE, 
                ROUND(SUM(AMOUNT),2) AS TOTAL_AMOUNT 
            FROM TRANSACTIONS 
            GROUP BY TXN_TYPE
            ORDER BY TOTAL_AMOUNT DESC""",
        "Q6" :
            """SELECT 
	            TXN_TYPE, 
            COUNT(STATUS) AS Failed_Count 
            FROM TRANSACTIONS WHERE STATUS ="failed"
            GROUP BY TXN_TYPE
            ORDER BY Failed_Count DESC""",
        "Q7" :
            """SELECT 
	            TXN_TYPE,
	            COUNT(TXN_TYPE) AS Total_Transactions
            FROM TRANSACTIONS
            GROUP BY TXN_TYPE
            ORDER BY Total_Transactions DESC""",
        "Q8" :
            """SELECT 
                T.CUSTOMER_ID,
                C.Name,
                COUNT(T.CUSTOMER_ID) AS High_Value_Transactions
            FROM TRANSACTIONS T
            JOIN CUSTOMERS C ON C.CUSTOMER_ID = T.CUSTOMER_ID 
            WHERE AMOUNT > 20000
            GROUP BY T.CUSTOMER_ID 
            HAVING COUNT(T.CUSTOMER_ID) >= 5
            ORDER BY High_Value_Transactions DESC""",
        "Q9" :
            """SELECT 
	            LOAN_TYPE, 
                ROUND(AVG(LOAN_AMOUNT),2) AS Loan_Amount, 
                ROUND(AVG(INTEREST_RATE),2) AS Interest_Rate 
            FROM LOANS
            GROUP BY LOAN_TYPE
            ORDER BY Interest_Rate DESC""",
        "Q10" :
            """SELECT 
                    CUSTOMER_ID,
                    COUNT(*) AS Active_Approved_Loans
                FROM LOANS 
                WHERE LOAN_STATUS IN ('ACTIVE','APPROVED')
                GROUP BY CUSTOMER_ID
                HAVING Active_Approved_Loans>1
                ORDER BY Active_Approved_Loans DESC;""",
        "Q11" :
            """SELECT 
	                CUSTOMER_ID 
                FROM LOANS 
                WHERE LOAN_STATUS NOT IN ('CLOSED')
                GROUP BY CUSTOMER_ID
                HAVING MAX(LOAN_AMOUNT) 
                ORDER BY MAX(LOAN_AMOUNT) desc
                LIMIT 5;""",
        "Q12" :
            """SELECT 
	                BRANCH, 
	                ROUND(AVG(LOAN_AMOUNT)) AS AVG_AMOUNT 
                FROM LOANS 
                GROUP BY BRANCH
                ORDER BY AVG_AMOUNT DESC;""",
        "Q13" :
            """SELECT 
	                CASE WHEN AGE BETWEEN 18 AND 25 THEN '18-25'
		                WHEN AGE BETWEEN 26 AND 35 THEN '26-35'
		                WHEN AGE BETWEEN 36 AND 45 THEN '36-45'
		                WHEN AGE BETWEEN 46 AND 55 THEN '46-55'
		                WHEN AGE BETWEEN 56 AND 69 THEN '56-69'
		                ELSE '70+'
	                END AS AGE_GROUP,
                    COUNT(CUSTOMER_ID) AS COUNT
                FROM CUSTOMERS
                GROUP BY AGE_GROUP
                ORDER BY AGE_GROUP""",
        "Q14" :
            """SELECT 
	                ISSUE_CATEGORY 
                FROM SUPPORT_TICK 
                GROUP BY ISSUE_CATEGORY
                ORDER BY AVG(DATEDIFF(DATE_CLOSED , DATE_OPENED)) DESC
                LIMIT 1;""",
        "Q15" :
            """SELECT 
	                SUPPORT_AGENT 
                FROM SUPPORT_TICK 
                WHERE STATUS = 'Resolved' AND PRIORITY ='CRITICAL' AND CUSTOMER_RATING >= 4
                GROUP BY SUPPORT_AGENT"""
    }

In [ ]:
def answers(que):
        result = quetions[que]
        return result

In [66]:
cursor.execute("USE BANK")
for i in range (1,16):
    answer = answers(f"Q{i}")
    df = pd.read_sql(answer,connection)
    print(f"------{i}----")
    print(df)
    print("\n")

------1----
                 CITY  Total_Customers  Average_Balance
0     Margaretchester                1        498739.50
1           Kinghaven                1        498317.41
2           Johnmouth                1        494779.97
3          East Robin                1        494699.91
4          Millerstad                1        484758.19
..                ...              ...              ...
482       Port Dakota                1          5128.16
483    West Ryanshire                1          3001.59
484         Juliaview                1          2647.41
485    North Dalefort                1          1862.66
486  Port Jenniferton                1          1669.73

[487 rows x 3 columns]


------2----
  ACCOUNT_TYPE
0      Current


------3----
  CUSTOMER_ID                Name  Total_Balance
0       C0314          Amy Nguyen      498739.50
1       C0011     Carolyn Higgins      498317.41
2       C0111        Lisa Russell      497040.59
3       C0392      Amber Williams     

C:\Users\Sushmethaa\AppData\Local\Temp\ipykernel_17992\1165803470.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(answer,connection)
C:\Users\Sushmethaa\AppData\Local\Temp\ipykernel_17992\1165803470.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(answer,connection)
C:\Users\Sushmethaa\AppData\Local\Temp\ipykernel_17992\1165803470.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(answer,connection)
C:\Users\Sushmethaa\AppData\Local\Temp\ipykernel_17992\1165803470.p